In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI
from env_loader import load_environment_variables
import os

env_path = '.env'
additional_env_vars = {"LANGCHAIN_PROJECT": "tagging-test"}
env_vars = load_environment_variables(env_path, additional_vars=additional_env_vars)

print(os.environ["LANGCHAIN_PROJECT"])

tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
"""
)


class Classification(BaseModel):
    sentiment: str = Field(description="The sentiment of the text")
    aggressiveness: int = Field(
        description="How aggressive the text is on a scale from 1 to 10"
    )
    language: str = Field(description="The language the text is written in")


# LLM
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0125").with_structured_output(
    Classification
)

tagging_chain = tagging_prompt | llm

tagging-test


In [9]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
tagging_chain.invoke({"input": inp})

Classification(sentiment='positive', aggressiveness=1, language='Spanish')

In [10]:
inp = "Weather is ok here, I can go outside without much more than a coat"
tagging_chain.invoke({"input": inp})

Classification(sentiment='positive', aggressiveness=2, language='English')